In [ ]:
subscription_id = "<INSERT-AZURE-SUBSCRIPTION-HERE>"
resource_group = "<INSERT-AZURE-RESOURCE-GROUP-HERE>"
workspace = "<INSERT-AZURE-ML-WORKSPACE-HERE>"

from azureml.core.model import Model
from azureml.core import Workspace


In [ ]:
ws = Workspace(subscription_id=subscription_id,
               resource_group=resource_group,
               workspace_name=workspace)

In [ ]:

# Register model
model = Model.register(ws, model_name="bart-text-summarization", model_path="bart-text-summarization/model")

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="clippy-summarize")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory="./bart-text-summarization/code",
    entry_script="./transformer_scorer.py",
)

In [ ]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [ ]:
service = Model.deploy(
    ws,
    "clippy",
    [model],
    dummy_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

In [ ]:
import requests
import json

uri = service.scoring_uri

headers = {"Content-Type": "application/json"}
data = {
    "query": "What color is the fox",
    "context": "The quick brown fox jumped over the lazy dog.",
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

In [ ]:
env = Environment(name='myenv')
python_packages = ['transformers', 'torch', 'tenserflow']
for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

inference_config = InferenceConfig(environment=env, source_directory='./bart-text-summarization', entry_script='code/echo.py')

In [ ]:
clippy = ml_client.online_deployments.get(name='summarize', endpoint_name="endpoint-clippy")
ml_client.online_deployments.begin_create_or_update(clippy)
